# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
from astroquery.splatalogue import Splatalogue  # We must acknowledge authors for use of this library
from astropy import units as u
from SpectralQuery import SpectralQuery
import pickle

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


# Data
The data comes from 4 text files.  Each file contains 2 columns of data, _frequency_ and _flux_.  We'll only need to read in the data if we don't already know which molecules could be contained within the frequencies (by running the code).

In [2]:
# read in the data
print("\n--- Reading in data ---")
filenames = ["./Data/Win0.clean1.contsub_Jy.rest.scom.c.txt",
             "./Data/Win1.clean1.contsub_Jy.rest.scom.c.txt",
             "./Data/Win2.clean1.contsub_Jy.rest.scom.c.txt",
             "./Data/Win3.clean1.contsub_Jy.rest.scom.c.txt"]
# 'data' will be a 4 element list, with each element representing the data from 1 text file
data = [np.loadtxt(f) for f in filenames]
generate_plot = True
print("--- Complete ---\n")

# Read in all_molecules if it exists
if os.path.isfile('./Data/all_molecules.csv'):
    print("\n--- Loading in all_molecules ---")
    all_molecules = pd.read_csv('./Data/all_molecules.csv', header = 0, index_col = 0).to_dict(orient = "index")
    generate_plot = False   # Whether or not to generate the plot
    print("--- Complete ---\n")

# read in spectral_lines if it exists   
if os.path.isfile('./Data/spectral_lines.pkl'):
    print("\n--- Loading in spectral_lines ---")
    with open('./Data/spectral_lines.pkl', 'rb') as file:
        spectral_lines = pickle.load(file)
    spectral_lines = {key: pd.DataFrame.from_dict(value, dtype = np.float64) for key, value in spectral_lines.items()}
    print("--- Complete ---\n")


--- Reading in data ---
--- Complete ---


--- Loading in all_molecules ---
--- Complete ---



## Functions

### create_plot()
This function will generate a figure with 4 axes and produce a plot of the frequency versus flux of the raw data.  It will then return the figure and handle to those axes.

In [3]:
def create_plot():
    """ Recreate the plot from Cordiner et. al. 2015, Figure 1 """
    # Defining the figure might need more finess if there are more than 4 datasets
    # We need to consult a domain expert to learn if ALMA data always comes back with 4 datasets
    fig, axs = plt.subplots(2, 2, figsize = (10, 6))
    for index, ax in enumerate(axs.flat):
        # Plot each data set
        ax.plot(data[index][:,0],
                data[index][:,1],
                linewidth = 0.25)
        ax.set(ylabel = "Flux (Jy)")
        ax.set(xlabel = "Frequency (GHz)")
        # Clean up the xticks
        ax.set_xticks(np.arange(round(data[index][0,0], 1), round(data[index][-1,0], 1), step = 0.5))
        # Remove the space from the borders of the plot along the X axis
        ax.autoscale(enable = True, axis = 'x', tight = True)
    # Add some space between the plots
    plt.subplots_adjust(hspace = .3, wspace = .3)
    return fig, axs

### add_lines(id, molecules)
This function adds dashed lines to a specified axis (`id`) at every frequency passed in by `molecules`.  `molecules` is a dictionary whose keys are the _frequencies_ above the 3$\sigma$ threshold and whose values are the _Chemical names_ of molecules that have a spectral line at that frequency, and also their _Molecule Tag_.  So each frequency will have a dictionary inside it with keys _Chemical Name_ and _Molecule Tag_ with values pulled from the results of a Splatalogue query.

In [4]:
def add_lines(id, molecules):
    """This function will add dashed lines to the plot where molecules were detected"""
    for freq in molecules.keys():
        axs[id].axvline(x = float(freq),
                        ymin = 0,
                        ymax = 1,
                        dashes = [18, 6],
                        linewidth = 0.25,
                        color = "gray",
                        alpha = 0.5)

### find_molecules()
This function constructs 2 dictionaries.  The first dictionary created is `molecules.`  `molecules` is a dictionary whose keys are the _frequencies_ above the 3$\sigma$ threshold and whose values are the _Chemical names_ of molecules that have a spectral line at that frequency, and also their _Molecule Tag_.  So each frequency will have a dictionary inside it with keys _Chemical Name_ and _Molecule Tag_ with values pulled from the results of a Splatalogue query.  `molecules` is unique to each axis on the figure.  That is, since there are 4 axes on the figure, `molecules` gets generated 4 times.  Each time those frequencies get passed to `add_lines(id, molecules)` along with the `id` of the axis that it is meant for.

The second dictionary is a global dictionary, `all_molecules`.  This dictionary's keys are the unique _Chemical Names_ identified from all 4 generations of the `molecules` dictionary.  The values are another dictionary with keys _Occurances_, _Molecule Tag_, and _Linelist_ whose values are the frequencies of spectral emissions this molecule was found at in the data, the molecule tag number, and the linelist representing what database the data was pulled from, respectively.

The `molecules` dictionary is generated by first identifying all frequencies where the flux exceeds the 3$\sigma$ threshold (where a flux value is greater than 3 standard deviations away from the mean).  Then, for each frequency that meets this criteria, a query to the Splatalogue API is made at that frequency $\pm$ some `delta` (initially set to 50kHz).  If the query returns any results, they are added to the `molecules` and `all_molecules` dictionarys.

In [5]:
def find_molecules():
    """ Classify the molecules from their frequency for each dataset """
    for id, dataset in enumerate(data):
        # Locate the indices where the flux is greater than 3 standard deviations
        # There are 4 datasets.  Column 0 is the frequency, column 1 is the flux
        # Splatalogue appears to be accurate up to 5 decimal places
        molecules = {}  # An empty dictionary to store the results of each detected molecule and rest frequency
        delta = 0.00005 # +/- when searching frequencies
        frequencies = np.round(dataset[np.where( dataset[:, 1] >= 3 * np.std(dataset[:, 1])), 0], 5)[0]
        for freq in frequencies:
            # Query Splatalogue at each frequency +/- delta using the CDMS and JPL databases
            results = Splatalogue.query_lines( (freq - delta)*u.GHz, (freq + delta)*u.GHz,
                                                show_molecule_tag = True,
                                                top20 = 'planet',
                                                line_lists = ['CDMS', 'JPL'],
                                                line_strengths = 'ls1')
            # Append the chemical names corresponding to the searched frequency.
            if len(results) > 0:
                molecules[freq] = {"Chemical Name": results["Chemical Name"].tolist(),
                                   "Molecule Tag": results["Molecule<br>Tag"].tolist()}
            else:
                molecules[freq] = {"Chemical Name": "Unknown",
                                   "Molecule Tag": None}
        
            # Append the chemical name and frequency to the dictionary of all molecules found
            if len(results) > 0:
                for i, molecule in enumerate(results["Chemical Name"].tolist()):
                    if molecule in all_molecules.keys():
                        all_molecules[molecule]["Occurances"].append(freq)
                    else:
                        # Add leading 0's to the Molecule Tag if it's less than 6 characters
                        molecule_tag = "0" + str(results["Molecule<br>Tag"][i]) if len(str(results["Molecule<br>Tag"][i])) < 6 else str(results["Molecule<br>Tag"][i])
                        # You might find a '-' for the first character.  If so, change it to a 0
                        if "-" in molecule_tag: molecule_tag = "0" + molecule_tag[1:]
                        # The molecule is new.  Add its tag, linelist, and frequency to all_molecules
                        all_molecules[molecule] = {"Molecule Tag": molecule_tag,
                                                   "Linelist": results["Linelist"][i],
                                                   "Occurances": [freq]}
            else:
                # If the query returns no results, then we've found a significant emmission frequency with
                # no known molecule associated with it.
                if "Unknown" in all_molecules.keys():
                    all_molecules["Unknown"]["Occurances"].append(freq)
                else:
                    all_molecules["Unknown"] = {"Molecule Tag": "None",
                                                "Linelist": "None",
                                                "Occurances": [freq]}
        add_lines(id, molecules)

### pull_spectral_lines(SpectralQuery, molecule_tag)

Once all the molecules that could be in the data have been identified, we can then query other databases (**CDMS** and **JPL**) to retrieve the complete set of every spectral line emmission frequency for each identified molecule.  To do this, we need to know what `molecule_tag` to search for along with the database (_linelist_) it came from.

The `SpectralQuery` library will do exactly this.  By supplying the _linelist_ to a `SpectralQuery` class object, and the `molecule_tag`, it will scrape the spectral emmisssion data from the appropriate web page and store it in a pandas data frame.  Since we don't know exactly how many queries we'll need to make, and since the number of rows each query returns is also unknown, the results are best saved in a dictionary for further usage.  The global dictionary `spectral_lines` will be constructed with this function.  Its keys will be the _Chemical Names_ from `all_molecules`, and its values will be the results returned from `SpectralQuery.getSpectralLines()`.

In [6]:
def pull_spectral_lines(spectral_lines):
    """Scrapes the given database for the spectral lines emmitted by the molecule with the specified molecule_tag"""
    print("Retrieving spectral line frequencies for molecules of interest...")
    for molecule in all_molecules.keys():
        # Viable Linelist options are "CDMS, JPL, or None."  If "None" (for "Unknown"), just pass
        if all_molecules[molecule]["Linelist"] == "CDMS":
            cdms.getSpectralLines(all_molecules[molecule]["Molecule Tag"])
            spectral_lines[molecule] = cdms.lines
        elif all_molecules[molecule]["Linelist"] == "JPL":
            results = jpl.getSpectralLines(all_molecules[molecule]["Molecule Tag"])
            spectral_lines[molecule] = jpl.lines
        else:
            continue
    spectral_lines = {key: pd.DataFrame.from_dict(value, dtype = np.float64) for key, value in spectral_lines.items()}
    return spectral_lines
    print("Done\n")

## Generate Queries

Set up global variables, preferences, and run the algorithm.

In [7]:
# Options
show_plot = False   # Whether or not to show the plot (doesn't seem to work in Jupyter Notebooks...)
show_output = False  # Whether or not to view the all_molecules dictionary
min_freq = data[0][0][0] * 1000  # The first frequency value in the data (in MHz)
max_freq = data[-1][-1][0] * 1000  # The last frequency value in the data (in MHz)

# Generte the plot if necessary
if generate_plot:
    fig, axs = create_plot()
    axs = axs.flat

# If all_molecules wasn't loaded in, generate and save it by running the find_molecules function
if "all_molecules" not in locals():
    all_molecules = {}  # This will store the molecule name and every frequency it is found at
    find_molecules()
    
    # Save the all_molecules dictionary
    pd.DataFrame.from_dict(all_molecules,
                           columns = ["Molecule Tag",
                                      "Linelist",
                                      "Occurances"],
                           orient = "index").to_csv("./Data/all_molecules.csv")
    
# If spectral_lines wasn't loaded in, generate and save it by running the pull_spectral_lines function
if "spectral_lines" not in locals():
    # Define the dictionary 'spectral_lines'
    spectral_lines = {}
    # Scrape the CDMS Database content
    print("Parsing the Cologne Database for Molecular Spectroscopy...")
    cdms = SpectralQuery("CDMS")
    print("Done\n")
    # Scrape the JPL Database content
    print("Parsing the Jet Propulsion Laboratory Molecular Spectroscopy Database...")
    jpl  = SpectralQuery("JPL")
    print("Done\n")
    # Iterate through each molecule in all_molecules to pull all their emmission frequencies
    spectral_lines = pull_spectral_lines(spectral_lines)
    # Discard entries that are outside the bounds of our data [min_freq, max_freq]
    for molecule in spectral_lines.keys():
        spectral_lines[molecule] = spectral_lines[molecule][(spectral_lines[molecule]["frequency"] >= min_freq) & 
                                                            (spectral_lines[molecule]["frequency"] <= max_freq)]
    # Save the spectral_lines dictionary
    with open('./Data/spectral_lines.pkl', 'wb') as file:
        pickle.dump(spectral_lines, file, protocol = pickle.HIGHEST_PROTOCOL)
    print("Done\n")

Parsing the Cologne Database for Molecular Spectroscopy...
Done

Parsing the Jet Propulsion Laboratory Molecular Spectroscopy Database...
Done

Retrieving spectral line frequencies for molecules of interest...


# Output
View whichever results were specified in the Main Script section.

In [8]:
# Not sure why this doesn't work properly in a Jupyter Notebook.  Might be due to %magic...
if show_plot:
    plt.show()

# Output all_molecules in an easily readable table format.
if show_output:
    print("{:<15} {:<15} {:<10} Occurances".format('Molecule','Molecule Tag','Linelist'))
    for molecule, values in all_molecules.items():
        tag = all_molecules[molecule]["Molecule Tag"]
        linelist = all_molecules[molecule]["Linelist"]
        occurances = all_molecules[molecule]["Occurances"]
        print("{}".format(molecule))
        print("{:<15} {:<15} {:<10} {}".format("", tag, linelist, occurances))

In [8]:
spectral_lines

{'&alpha;-Alanine':          frequency  uncertainty  intensity
 12215  221492.0956       0.2198    -4.3549
 12216  221524.3642       0.2059    -4.6464
 12217  221532.2499      19.6553    -5.0812
 12218  221532.2499      19.6553    -5.0812
 12219  221695.6921       0.5233    -4.3325
 12220  221695.6938       0.5233    -4.3325
 12221  221743.5429       0.7118    -3.3188
 12222  221743.5429       0.7118    -3.3188
 12223  221770.9567       0.6261    -4.6949
 12224  221779.7090       1.4265    -4.8327
 12225  221793.5760       0.2160    -3.0753
 12226  221817.0912       0.5008    -3.0267
 12227  221817.0912       0.5008    -4.2331
 12228  221817.0912       0.5008    -3.0267
 12229  221817.0913       0.5008    -4.2331
 12230  221981.4345      52.0539    -4.7948
 12231  221981.4345      52.0539    -4.7948
 12232  222039.2276       0.3487    -4.7294
 12233  222039.6432      35.2680    -4.9583
 12234  222039.6432      35.2680    -4.9583
 12235  222046.2462       7.0633    -5.1819
 12236  22204